<img src='https://www.reliance-project.eu/wp-content/uploads/2021/03/Asset-8mdpi.svg' alt='Layout' align='left' width='25%'></img> 
<img src='https://media-exp1.licdn.com/dms/image/C560BAQFFDze0s3l-pQ/company-logo_200_200/0?e=2159024400&v=beta&t=jG_sh9NpwE6yc2TspIcL2YUR2X7OBzTrRSu6w7sNYtc' alt='Layout' align='right' width='10%'></img> 

# Using CAMS European air quality analysis from Copernicus Atmosphere Monitoring with RELIANCE services

## Analysis over Norway

<div class="alert alert-success alert-info">
<b>How to discover RELIANCE datacube resources (spatial and temporal search and subsetting)</b></div>

This notebook shows how to discover and access the [Copernicus Atmosphere Monitoring](https://ads.atmosphere.copernicus.eu/#!/home) products available in the **RELIANCE** datacube resources, by using the functionalities provided in the <font color='blue'> **Adam API** </font>. The process is structured in 6 steps, including example of data analysis and visualization with the Python libraries installed in the Jupyter environment.

- [1. Authentication](#1.Authentication)
- [2. Datasets Discovery](#2.Datasets_Discovery)
- [3. Products Discovery](#3.Products_Discovery)
- [4. Data Access](#4.Data_Access)
- [5. Data Analysis and Visualizarion](#5.Data_Analysis_Visualization)warnings.filterwarnings('ignore')

In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

## <a id=1.Authentication></a> **Step 1: Authentication** 

The following lines of code will show the personal **Adam API-Key** of the user and the endpoint currently in use, that provide access to the products in the related catalogue. At the end of the execution, if the authentication process is successfull the personal token and the expiration time should be returned as outputs.

In [3]:
!pip install adamapi

In [4]:
import adamapi as adam
a = adam.Auth()
a.setKey('d2rPsRgBMM29mdFRP96juOvdPzg4ku2vwWEvnsBWXAM')
a.setAdamCore('https://reliance.adamplatform.eu')
a.authorize() 

{'expires_at': '2021-10-06T19:27:18.401Z',
 'access_token': '9c5d09dedafe42eca1b4d03d645ddbaf',
 'refresh_token': 'a9cab12f1c4e4dbc9d480cb38150f66e',
 'expires_in': 3600}

## <a id=2.Datasets_Discovery></a> **Step 2: Datasets Discovery**

After the authorization, the user can browse the whole catalogue, structured as a JSON object after a pagination process,  displaying all the available datasets. This operation can be executed with the <font color='blue'> **getDatasets()** </font> function without including any argument. Some lines of code should be added to parse the Json object and extract the names of the datasets.The Json object can be handled as a Python dictionary.

### Pre-filter datasets

We will discover all the available datasets in the ADAM platform but will only print elements of interest **EU_CAMS** e.g. [European air quality datasets](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) from Copernicus Atmosphere Monitoring Service

In [5]:
from adamapi import Datasets

datasets = Datasets(a)
catalogue = datasets.getDatasets()

#Extracting the size of the catalogue

total = catalogue['properties']['totalResults']
items = catalogue['properties']['itemsPerPage']
pages = total//items

print('----------------------------------------------------------------------')
print('\033[1m' + 'List of available datasets:')
print ('\033[0m')

#Extracting the list of datasets across the whole catalogue

for i in range(0,pages):
    page = datasets.getDatasets(page = i)
    for element in page['content']: 
        if 'EU_CAMS' in element['title'] :
            print(element['title'] + "\033[1m" + " --> datasetId "+ "\033[0m" + "= " + element['datasetId'])

----------------------------------------------------------------------
List of available datasets:

EU_CAMS_SURFACE_C2H3NO5_G --> datasetId = 69619:EU_CAMS_SURFACE_C2H3NO5_G
EU_CAMS_SURFACE_CO_G --> datasetId = 69620:EU_CAMS_SURFACE_CO_G
EU_CAMS_SURFACE_NH3_G --> datasetId = 69621:EU_CAMS_SURFACE_NH3_G
EU_CAMS_SURFACE_NMVOC_G --> datasetId = 69622:EU_CAMS_SURFACE_NMVOC_G
EU_CAMS_SURFACE_NO2_G --> datasetId = 69623:EU_CAMS_SURFACE_NO2_G
EU_CAMS_SURFACE_NO_G --> datasetId = 69624:EU_CAMS_SURFACE_NO_G
EU_CAMS_SURFACE_O3_G --> datasetId = 69625:EU_CAMS_SURFACE_O3_G
EU_CAMS_SURFACE_PM10_G --> datasetId = 69626:EU_CAMS_SURFACE_PM10_G
EU_CAMS_SURFACE_PM25_G --> datasetId = 69627:EU_CAMS_SURFACE_PM25_G
EU_CAMS_SURFACE_REC_G --> datasetId = 69628:EU_CAMS_SURFACE_REC_G
EU_CAMS_SURFACE_SIA_G --> datasetId = 69629:EU_CAMS_SURFACE_SIA_G
EU_CAMS_SURFACE_SO2_G --> datasetId = 69630:EU_CAMS_SURFACE_SO2_G
EU_CAMS_SURFACE_TEC_G --> datasetId = 69631:EU_CAMS_SURFACE_TEC_G


We are interested by **Particulate matter < 10 µm (PM10)** so we will discover **EU_CAMS_SURFACE_PM10_G** and print the metadata of this particular dataset, showing the data provenance.

In [6]:
datasetID = '69626:EU_CAMS_SURFACE_PM10_G'

print('\033[1;34m' + 'Metadata of ' + datasetID + ':')
print ('\033[0;0m')

paged = datasets.getDatasets(datasetID)
for i in paged.items():
    print("\033[1m" +  str(i[0]) + "\033[0m" + ': ' + str(i[1]))

Metadata of 69626:EU_CAMS_SURFACE_PM10_G:

datasetId: 69626:EU_CAMS_SURFACE_PM10_G
creationDate: 2021-07-12T02:00:00Z
dataType: Float32
epsg: 4326
keywords: []
license: {'documentationUrl': '', 'dataProviderName': 'ADS', 'dataProviderUrl': '', 'licenseId': '', 'dataPolicy': '', 'doi': '', 'credits': ''}
maxValue: [2645.995849609375]
minValue: [0.003789698239415884]
numberOfRecords: 56702
profile: {'profileSchema': 'eo_profile_schema.json', 'name': 'Earth Observation', 'mission': 'CAMS', 'sensor': 'CAMS', 'processingLevel': 'forecast', 'instrument': '', 'platform': ''}
resolutionUnit: degree
temporalResolution: Hourly
unit: 
unitDescription: 
updateDate: 2021-10-06T18:01:56Z
geometry: {'type': 'Polygon', 'coordinates': [[[-25.000012, 29.999997], [44.999988, 29.999997], [44.999988, 71.999997], [-25.000012, 71.999997], [-25.000012, 29.999997]]]}
resolutions: [0.1]
anyText: 4326,ESA,Daily,ESA
applications: ['Atmosphere']
datasetManager: govoni@meeo.it
datasetManagerOrganisation: meeo
descr

## <a id=3.Products_Discovery></a> **Step 3: Products Discovery**

The products discovery operation related to a specific dataset is implemented in the Adam API with the <font color='blue'> **getProducts()** </font> function. A combined **spatial and temporal search** can be requested by specifying the <font color='red'> **datasetId** </font> for the selected dataset,the <font color='red'> **geometry** </font> argument that specifies the <u>Area Of Interest</u> and a temporal range, defined by `startDate` and `endDate` . The geometry must **<u>always</u>** be defined by a <font color='red'> **GeoJson object** </font> that describes the polygon in the **<u>counterclockwise winding order**</u>. The optional arguments `startIndex` and `maxRecords` can set the list of the results returned as an output. The results of the search are displayed with their metadata and they are sorted starting from the most recent product.

### Generate geometry for a given country
The geometry field is extracted from a GeoJSON object , retrieving the value of the "feature" element.


In [7]:
!wget https://raw.githubusercontent.com/mledoze/countries/master/data/nor.geo.json

--2021-10-06 18:27:58--  https://raw.githubusercontent.com/mledoze/countries/master/data/nor.geo.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180319 (176K) [text/plain]
Saving to: ‘nor.geo.json.1’

nor.geo.json.1      100%[===================>] 176.09K  --.-KB/s    in 0.02s   

2021-10-06 18:27:58 (9.94 MB/s) - ‘nor.geo.json.1’ saved [180319/180319]



### Search data

In [8]:
!pip install geojson_rewind

In [9]:
from adamapi import Search
from geojson_rewind import rewind
import json

The GeoJson object needs to be rearranged according to the counterclockwise winding order.This operation is executed in the next few lines to obtain 
a geometry that meets the requirements of the method. **Geom_1** is the final result to be used in the discovery operation.

In [10]:
with open('nor.geo.json') as f:
    geom_dict = json.load(f)
output = rewind(geom_dict)    
geom_1 = str(geom_dict['features'][0]['geometry'])

Copernicus air quality analyses are hourly product but when we select a given date, we will only get the first 10 products. 
Below, we make a list of the first 10 available products for the 1st September 2020 e.g. we restrict our search to this date.

In [11]:
start_date = '2019-09-01'
end_date = '2019-09-01'

In [12]:
search = Search( a )
results = search.getProducts(
    datasetID, 
    geometry= geom_1,
    startDate = start_date,
    endDate = end_date
 )

# Printing the results

print('\033[1m' + 'List of available products:')
print ('\033[0m')
count = 1
for i in results['content']:

        print("\033[1;31;1m" + "#" + str(count))
        print ('\033[0m')
        for k in i.items():
            print(str(k[0]) + ': ' + str(k[1]))
        count = count+1
        print('------------------------------------')

AdamApiError: Internal Server Error

## <a id=4.Data_Access></a> **Step 4: Data Access**

After the data discovery operation that retrieves the availability of products in the catalogue, it is possible to access the data with the <font color='blue'> **getData** </font> function. Each product in the output list intersects the selected geometry and the following example shows how to access a specific product from the list of results obtained in the previous step. While the <font color='red'> **datasetId** </font> is always a mandatory parameter, for each data access request the <font color='blue'> **getData** </font> function needs only one of the following arguments: <font color='red'> **geometry** </font> or <font color='red'> **productId** </font>, that is the value of the <font color='blue'> **_id** </font> field in each product metadata. In the case of a <u>**spatial and temporal search**</u> the geometry must be provided to the function, together with the time range of interest. 
The output of the <font color='blue'> **getData** </font> function is <u>always</u> a <font color='red'> **.zip** </font> file containing the data retrieved with the data access request, providing the spatial **subset** of the product. The zip file will contain a geotiff file for each of the spatial subsets extracted in the selected time range.

In [13]:
from adamapi import GetData

#### Define a function to select a time range and get data

In [14]:
def getZipData(auth, dataset_info):
    data=GetData(auth)
    image = data.getData(
    datasetId = dataset_info['datasetID'],
    startDate = dataset_info['startDate'],
    endDate = dataset_info['endDate'],
    geometry = dataset_info['geometry'],
    outputFname = dataset_info['outputFname'])
    print(image)

#### Get PM10 for each day of September 2019, 2020 and 2021 (time 00:00:00)

This process can take a bit of time so be patient!

In [15]:
import time
from IPython.display import clear_output

In [ ]:
start = time.time()

for year in ['2019', '2020', '2021']:
    datasetInfo = {
    'datasetID' : '69626:EU_CAMS_SURFACE_PM10_G',
    'startDate' : year + '-09-01',
    'endDate' : year + '-09-30',
    'geometry' : geom_1,
    'outputFname' : './PM10_NOR_ADAMAPI_' + year + '.zip'
    }
    getZipData(a, datasetInfo)
    
end = time.time()
clear_output(wait=True)
delta1 = end - start
print('\033[1m'+'Processing time: ' + str(round(delta1,2)))

AF  {'type': 'MultiPolygon', 'coordinates': [[[[9.4858320000001, 42.615273], [9.49472, 42.603607], [9.4827770000001, 42.613052], [9.47778, 42.617218], [9.465277, 42.630829], [9.457777, 42.643326], [9.4858320000001, 42.615273]]], [[[9.446665, 42.67889], [9.4480550000001, 42.64944], [9.452221, 42.630272], [9.473888, 42.582222], [9.47805, 42.576111], [9.50555, 42.563889], [9.509998, 42.563606], [9.51139, 42.56721], [9.511665, 42.571663], [9.509443, 42.578049], [9.503054, 42.59166], [9.497221, 42.60083], [9.50028, 42.59861], [9.5202770000001, 42.572495], [9.531666, 42.54916], [9.5338880000001, 42.541939], [9.562222, 42.272774], [9.5599990000001, 42.19221], [9.5555550000001, 42.127777], [9.5533330000001, 42.115555], [9.54583, 42.102219], [9.4480550000001, 41.999443], [9.42555, 41.975], [9.41111, 41.954163], [9.405554, 41.934998], [9.397192, 41.875931], [9.396666, 41.862778], [9.398611, 41.85083], [9.402498, 41.840271], [9.404444, 41.828331], [9.404165, 41.81916], [9.398888, 41.698883], [9.3

## <a id=5.Data_Analysis_Visualization></a> **Step 5: Data Analysis and Visualization**

The data retrieved via the Adam API are now available as a zip file that must be unzipped to directly handle the data in a geotiff format. Then with the Python packages provided in the Jupyter environment it is possible to process and visualized the requested product.

#### Unzip data

In [ ]:
import pathlib
import zipfile

In [ ]:
def unzipData(filename):
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(path = pathlib.Path(filename).stem)

In [ ]:
for year in ['2019', '2020', '2021']:
    unzipData('./PM10_NOR_ADAMAPI_' + year + '.zip')

#### Read data and make a monthly average

In [ ]:
import xarray as xr
import glob

We now read these files using `xarray`. First, we make a list of all the geotiff files in a given folder. To ensure each raster is labelled correctly with its time, we can use a helper function `paths_to_datetimeindex()` to extract time information from the file paths we obtained above. We then load and concatenate each dataset along the time dimension using `xarray.open_rasterio()`, convert the resulting `xarray.DataArray` to a `xarray.Dataset`, and give the variable a more useful name (**PM10**)

In [ ]:
from datetime import datetime

In [ ]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('_')[-1].split('.')[0], '%Y-%m-%dt%f') for date in paths]

In [ ]:
def getData(dirtif):
    geotiff_list = glob.glob(dirtif)
    # Create variable used for time axis
    time_var = xr.Variable('time', paths_to_datetimeindex(geotiff_list))
    # Load in and concatenate all individual GeoTIFFs
    geotiffs_da = xr.concat([xr.open_rasterio(i, parse_coordinates=True) for i in geotiff_list],
                        dim=time_var)
    # Covert our xarray.DataArray into a xarray.Dataset
    geotiffs_da = geotiffs_da.to_dataset('band')
    # Rename the dimensions to make it CF-convention compliant
    geotiffs_da = geotiffs_da.rename_dims({'y': 'latitude', 'x':'longitude'})
    # Rename the variable to a more useful name
    geotiffs_da = geotiffs_da.rename_vars({1: 'PM10', 'y':'latitude', 'x':'longitude'})

    return geotiffs_da

In [ ]:
geotiff_ds = getData('./PM10_NOR_ADAMAPI_20*/*.tif')
geotiff_ds['PM10'].attrs = {'units' : 'µg m-3', 'long_name' : 'Particulate matter < 10 µm'}
geotiff_ds

#### Analyze data

Make yearly average for September month

In [ ]:
geotiff_dm = geotiff_ds.groupby('time.year').mean('time', keep_attrs=True)

In [ ]:
geotiff_dm

#### Visualize data

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cmaps

In [ ]:
# generate figure
proj_plot = ccrs.Mercator(central_longitude=24.0)

lcmap = cmaps.BlueYellowRed
# Only plot values greater than 0
p = geotiff_dm['PM10'].where(geotiff_dm['PM10'] > 0).plot(x='longitude', y='latitude',transform=ccrs.PlateCarree(),
                            subplot_kws={"projection": proj_plot},
                            size=8,
                            col='year', col_wrap=3, robust=True, cmap=lcmap, add_colorbar=True)

# We have to set the map's options on all four axes
for ax,i in zip(p.axes.flat,  geotiff_dm.year.values):
    ax.coastlines()
    ax.set_title('Surface PM10\n' + 'September ' + str(i), fontsize=10)

plt.savefig('PM10_september_NOR_2019-2021.png')

#### Plot one single date

In [ ]:
fig=plt.figure(figsize=(10,10))
# Define the projection
crs=ccrs.PlateCarree()

# We're using cartopy and are plotting in Orthographic projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.Mercator(central_longitude=12.0))
ax.coastlines(resolution='10m')

# custom colormap

lcmap = cmaps.BlueYellowRed

# We need to project our data to the new Mercator projection and for this we use `transform`.
# we set the original data projection in transform (here PlateCarree)
# we only plot values greather than 0
img = geotiff_ds['PM10'].where(geotiff_ds['PM10'] > 0).sel(time='2021-09-15').plot(ax=ax, transform=ccrs.PlateCarree(), cmap=lcmap)  

# Title for plot
plt.title('Surface PM10 \n 15th September 2021 over Norway',fontsize = 16, fontweight = 'bold', pad=10)
plt.savefig('PM10_september_NOR_2021-09-15.png')

#### Interactive plot with bokeh

In [ ]:
import hvplot.xarray

<div align="center">
    <h1 style="font-size:5vw">PM10 over Norway from 1st September to 30th September 2021</h1>
</div>

In [ ]:
geotiff_ds.hvplot.labels( text_font_size='6pt', text_color='blue').opts(xoffset=20)
geotiff_ds.where(geotiff_ds['PM10'] > 0).sel(time=slice('2021-09-01', '2021-09-30')).PM10.hvplot(
    groupby="time",
    cmap=lcmap,
    geo=True,
    coastline='10m',
    frame_width=400,
    clim=(0,35) 
)

#### Plot a timeseries

<div align="center">
    <h1 style="font-size:5vw">PM10 over Lillehammer from 1st September to 30th September 2021</h1>
</div>

Sort times and then select one location

In [ ]:
latitude = 61.1153
longitude = 10.4662

In [ ]:
geotiff_ds = geotiff_ds.sortby('time')
ts = geotiff_ds['PM10'].where(geotiff_ds['PM10'] > 0).sel(latitude=latitude, longitude=longitude, method='nearest').sel(time=slice('2021-09-01', '2021-09-30'))

In [ ]:
ts.hvplot(color='purple') * ts.hvplot.scatter(marker='o', color='blue', size=15)